### **Imports and data fetching**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import wandb
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
import itertools
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from torch.optim import RMSprop

#### **Decision about graph printing**

In [ ]:
graphs = input("Print graphs? [y/n]: ").lower() == 'y'

In [ ]:
# fetch dataset
magic_gamma_telescope = fetch_ucirepo(id=159)

# data (as pandas dataframes)
DS_feature = magic_gamma_telescope.data.features
DS_target = magic_gamma_telescope.data.targets

# metadata
print(magic_gamma_telescope.metadata)

# variable information
print(magic_gamma_telescope.variables)



### yay

#### **Data exploration**

In [ ]:
#data exloration
print(DS_feature.describe())

print("-------------------------------------------------------------")

print(DS_target.describe())


##### **Duplicates detection**

In [ ]:
print("Duplicates in features:", DS_feature.duplicated().sum())

#DS_feature = DS_feature.drop_duplicates()


##### **Anomalies detection and handling**

In [ ]:
#anomalies detected
def count_anomalies(df, column, normal_min, normal_max):
    mask_normal = (df[column] >= normal_min) & (df[column] <= normal_max)
    anomalies = (~mask_normal).sum()
    return anomalies

print(f"fLength: ", count_anomalies(DS_feature, 'fLength', 0, 1000))
print(f"fWidth: ", count_anomalies(DS_feature, 'fWidth', 0, 1000))
print(f"fSize: ", count_anomalies(DS_feature, 'fSize', 0, 1000))
print(f"fAlpha: ", count_anomalies(DS_feature, 'fAlpha', 0, 180))
print(f"fDist: ", count_anomalies(DS_feature, 'fDist', 0, 1000))


##### **Outlier detection and handling**

In [ ]:
features = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist']


for col in features:
    Q1 = DS_feature[col].quantile(0.25)
    Q3 = DS_feature[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # set outliers to NaN

    DS_feature[col] = DS_feature[col].where(DS_feature[col].between(lower, upper), np.nan)
    #DS_feature.loc[(DS_feature[col] < lower) | (DS_feature[col] > upper), col] = np.nan


# --- Missingness and outlriers handling ---
threshold = 30

missing_pct = DS_feature.isna().mean() * 100
to_drop_cols = missing_pct[missing_pct > threshold ].index.tolist()
to_impute_cols = missing_pct[(missing_pct >= 0.1) & (missing_pct <= threshold)].index.tolist()

print(f'Missing percentages:\n{missing_pct}')
print(f'Number of outliers: {(DS_feature.isna().sum().sum())}')

dropped_cols = []
imputed_cols = []

# drop columns with more than 30% (threshold) missing values
for col in to_drop_cols:
    if col in DS_feature.columns:
        DS_feature = DS_feature.drop(columns=[col])
        dropped_cols.append(col)

# for columns with 0.1-30% missing -> impute

for col in to_impute_cols:
    if col not in DS_feature.columns:
        continue
    # checks if the column’s dtype is numeric
    if pd.api.types.is_numeric_dtype(DS_feature[col]):
        median_val = DS_feature[col].median(skipna=True)
        # replaces missing values with the median (less sensetive to outliners than mean)
        # inplace=True means the changes are applied directly to the DataFrame
        DS_feature[col].fillna(median_val, inplace=True)
        imputed_cols.append((col, "median", median_val))
    else:
        # for non-numeric, impute with mode if available
        try:
            mode_val = DS_feature[col].mode(dropna=True)
            if len(mode_val) > 0:
                mode_val = mode_val.iloc[0]
                DS_feature[col].fillna(mode_val, inplace=True)
                imputed_cols.append((col, "mode", mode_val))
            else:
                # if no mode, fill with placeholder
                DS_feature[col].fillna("missing", inplace=True)
                imputed_cols.append((col, "placeholder", "missing"))
        except Exception as e:
            DS_feature[col].fillna("missing", inplace=True)
            imputed_cols.append((col, "placeholder", "missing"))

In [ ]:
print("Missingness handling summary:")
print(f'Dropped columns (missingness > {threshold}%): {len(dropped_cols)}\n{dropped_cols}')
print(f'Imputed columns (missingness between 5% and {threshold}%): {len(imputed_cols)}\n{imputed_cols}')

###### **QQ plot and hist plot for normal distribution check**

In [ ]:
features = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist']
if graphs:
    fig, axes = plt.subplots(2, 5, figsize=(15,8))

    for ax, feature in zip(axes.flatten(), features):
        stats.probplot(DS_feature[feature], dist="norm", plot=ax)
        ax.set_title(f"QQ Plot: {feature}", fontsize=8)

    plt.tight_layout()
    plt.show()
else:
    print('no')

In [ ]:
if graphs:
    fig, axes = plt.subplots(2, 5, figsize=(15,8))

    for ax, feature in zip(axes.flatten(), features):
        sns.histplot(DS_feature[feature], kde=True, ax=ax)
        ax.set_title(f"Distribution of {feature} by class")


    plt.tight_layout()
    plt.show()
else:
    print("No plotting, just scheming")

###### **Boxplot for outlier check**

In [ ]:
if graphs:
    DS_feature.plot(kind='box', subplots=True,
                                  layout=(2, 5), figsize=(15, 10), sharex=False)
    plt.suptitle("Boxplots for all features")
    plt.show()
else:
    print("EE")

##### **Class distribution**

In [ ]:
if DS_target['class'].dtype == 'object':
        DS_target['class'] = DS_target['class'].map({'g': 1, 'h': 0})

if graphs:

    plt.figure(figsize=(15, 8))
    sns.countplot(x='class', data=DS_target, palette='Set2', hue='class')
    plt.title("Distribution of Classes (Gamma vs Hadron)")
    plt.xlabel("Class")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

else:
    print("nah")

##### **Corelation Matrix**

In [ ]:
if graphs:
    df = DS_feature.copy()
    df['class'] = DS_target

    df.drop_duplicates()

    if df['class'].dtype == 'object':
        df['class'] = df['class'].map({'g': 1, 'h': 0})

    corr_matrix = df.corr(numeric_only=True)

    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title("Correlation Matrix - MAGIC Gamma Telescope Dataset", fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("Dont feel like it!")

#### **Data transformation and split**

In [ ]:
DS_feature_scaled = DS_feature.copy()

imputer = SimpleImputer(strategy='median')
DS_feature = pd.DataFrame(imputer.fit_transform(DS_feature), columns=DS_feature.columns)

cols_power  = ['fLength', 'fWidth', 'fSize', 'fAlpha','fAsym', 'fM3Long', 'fM3Trans']

pt = PowerTransformer(method='yeo-johnson')

DS_feature_scaled[cols_power] = pt.fit_transform(DS_feature[cols_power])

##### **Distribution check after the transformation**

In [ ]:
if graphs:
    fig, axes = plt.subplots(2, 5, figsize=(15,8))

    for ax, feature in zip(axes.flatten(), features):
        sns.histplot(DS_feature_scaled[feature], kde=True, ax=ax)
        ax.set_title(f"Distribution of {feature} by class")


    plt.tight_layout()
    plt.show()
else:
    print("Meeh!")

In [ ]:
if graphs:
    fig, axes = plt.subplots(2, 5, figsize=(15,8))

    for ax, feature in zip(axes.flatten(), features):
        stats.probplot(DS_feature_scaled[feature], dist="norm", plot=ax)
        ax.set_title(f"QQ Plot: {feature}", fontsize=8)

    plt.tight_layout()
    plt.show()
else:
    print("Think abut the gas prices!")

##### **Train/Test split**

In [ ]:
X = DS_feature_scaled
y = DS_target["class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=14, stratify=y
)

#### **Feature importance**

In [ ]:


remaining_feat = list(features)
selected_feat = []
current_score = 0.0
best_new_score = 0.0

progress = []

while remaining_feat:
    score_with_candidates = []

    for candidate in remaining_feat:
        model = LinearRegression()
        model.fit(X_train[selected_feat + [candidate]], y_train)
        score = model.score(X_test[selected_feat + [candidate]], y_test)
        score_with_candidates.append((score, candidate))

    score_with_candidates.sort()
    best_new_score, best_candidate = score_with_candidates.pop()

    if current_score < best_new_score:
        remaining_feat.remove(best_candidate)
        selected_feat.append(best_candidate)
        current_score = best_new_score
        # print(f"Adding {best_candidate} with score {current_score}")

        progress.append((len(selected_feat), best_candidate, current_score))

    else:
        break

print(f"Forward Selection Result: {selected_feat};")
print(f"Number of Selected Features: {len(selected_feat)};")
print(f"Final Validation Score: {current_score:.4f}")


##### **Forward selection R^2**

In [ ]:
if graphs:
    steps, feats, scores = zip(*progress)
    plt.figure(figsize=(8,4))
    plt.plot(steps, scores, marker='o')
    plt.title("Forward Selection Performance")
    plt.xlabel("Number of Selected Features")
    plt.ylabel("Validation R²")
    plt.grid(True)
    plt.show()
else:
    print("No Forward Selection Result!")

In [ ]:
remaining_feat = list(X_train.columns)
current_score = 0.0

model = LinearRegression()
model.fit(X_train[remaining_feat], y_train)
best_score = model.score(X_test[remaining_feat], y_test)

scores_progress = [(len(remaining_feat), best_score)]
print(f"Initial R² with all features: {best_score:.4f}")

##### **Backward selection R^2**

In [ ]:
while len(remaining_feat) > 1:
    scores_with_candidates = []
    
    for feature in remaining_feat:
        reduced_features = [f for f in remaining_feat if f != feature]
        model = LinearRegression()
        model.fit(X_train[reduced_features], y_train)
        score = model.score(X_test[reduced_features], y_test)
        scores_with_candidates.append((score, feature))
    
    scores_with_candidates.sort(reverse=True)
    best_new_score, worst_feature = scores_with_candidates[0]

    if best_new_score < best_score - 0.001:
        break

    remaining_feat.remove(worst_feature)
    best_score = max(best_new_score, best_score)
    scores_progress.append((len(remaining_feat), best_new_score))
    print(f"Removed {worst_feature}, new R²: {best_new_score:.4f}")

print("Backward Elimination Result:")
print(f"Selected features: {remaining_feat};")
print(f"Number of selected features: {len(remaining_feat)}")
print(f"Final Validation R²: {best_score:.4f}")

In [ ]:
if graphs:
    steps, scores = zip(*scores_progress)
    plt.figure(figsize=(8,4))
    plt.plot(steps, scores, marker="o", linestyle="-", color="brown")
    plt.title("Backward Elimination Performance")
    plt.xlabel("Number of Remaining Features")
    plt.ylabel("Validation R²")
    plt.gca().invert_xaxis()
    plt.grid(True)
    plt.show()
else:
    print("Nothing to do!")

#### **Trainning setup**

In [ ]:
#choose what to run
run = int(input(
    "Run config (enter number): "
    "[0] default training; [1] early stopping; [2] grid search; [3] grid + early stopping\n"
))

In [ ]:
#single run params

test_split = 0.2
batch_size = 512
random_state = 14
in_features = None # later int the following cells is split where it sets in features auto (no need 2 set)
learning_rate = 1e-3
epochs = 500
dropout_rate = 0.2
weight_decay = 1e-4
pos = 1.8


In [ ]:
# grid search config

config = {
    "lr": [0.001, 5e-4],
    "hidden_dim": [64, 128],
    "dropout_p": [0.05, 0.1],
    "epochs": [10],
    "batch_size": [256, 512],
    "weight_decay": [1e-5, 1e-3],
    "pos_weight": [1,0.8,2]
}

# early stopping settings

monitor = "accuracy"
mode = "max"
patience = 50 # epochs
best_metric = -float("inf") if mode == "max" else float("inf")
wait = 0

# best model path for later possibility of loading it back

best_model_path = "best_model.pt"


#### **Feature setting and final split**

In [ ]:
T_features = DS_feature[['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fAlpha', 'fDist']].values
T_labels = DS_target['class'].values

in_features = T_features.shape[1] #here is the in feature set from above initialization


X_train, X_val, y_train, y_val = train_test_split(
    T_features, T_labels, test_size=test_split, random_state=random_state
)


X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)


train_ds = TensorDataset(X_train, y_train)
val_ds = TensorDataset(X_val, y_val)


#### **Loading data**

In [ ]:
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

#### **Model structure**

In [ ]:
class MagicalNet(nn.Module):
    def __init__(self, in_dim, hidden_dim=128, dropout_p=dropout_rate):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.drop1 = nn.Dropout(dropout_p)

        self.fc2 = nn.Linear(hidden_dim, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.drop2 = nn.Dropout(dropout_p)

        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.fc1(x)))
        x = self.drop1(x)
        x = F.leaky_relu(self.bn2(self.fc2(x)))
        x = self.drop2(x)
        return self.fc3(x)


In [ ]:

model = MagicalNet(in_dim=in_features, hidden_dim=64, dropout_p=dropout_rate)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos]))
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.001)

In [ ]:
wandb.login()

In [ ]:
if run == 0 or run == 1:
    wandb.init(project="ZNEUS_R&B", name="gamma_classification",
        config={
            "epochs": epochs,
            "batch_size": batch_size,
            "learning_rate": learning_rate,
            "dropout_p": dropout_rate,
            "in_features": in_features,
            "weight_decay": weight_decay,
            "architecture": "MagicalNet"
        })

In [ ]:
if run == 0:
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for xb, yb in train_loader:
            xb, yb = xb, yb.unsqueeze(1)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)

        epoch_loss = running_loss / len(val_loader.dataset)
        #scheduler.step()
        #print(f"Current LR: {optimizer.param_groups[0]['lr']}")
        wandb.log({"train_loss": epoch_loss, "epoch": epoch + 1})

        print(f"Epoch {epoch+1}/{epochs} — loss: {epoch_loss:.4f}")
else:
    print("Not me!")

In [ ]:
if run == 0:
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for xb, yb in val_loader:
            logits = model(xb)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).long()
            all_preds.append(preds.cpu())
            all_targets.append(yb.unsqueeze(1).long())

    all_preds = torch.cat(all_preds).squeeze().numpy()
    all_targets = torch.cat(all_targets).squeeze().numpy()
    accuracy = (all_preds == all_targets).mean()

    precision = precision_score(all_targets, all_preds)
    recall = recall_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds)
    cm = confusion_matrix(all_targets, all_preds)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print("Confusion matrix:\n", cm)

    wandb.log({
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion matrix": cm,
    })
else:
    print("Not me either!")


In [ ]:
if run == 1:
    for epoch in range(epochs):

        # -------- training --------

        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb, yb.unsqueeze(1)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)

        #scheduler.step()
        train_epoch_loss = running_loss / len(train_loader.dataset)
        wandb.log({"train_loss": train_epoch_loss, "epoch": epoch + 1})
        #print(f"Current LR: {optimizer.param_groups[0]['lr']}")

        # -------- validation / evaluation --------

        model.eval()
        probs_list = []
        targets_list = []
        with torch.no_grad():
            for xb, yb in val_loader:
                logits = model(xb)
                probs = torch.sigmoid(logits).cpu().numpy().squeeze()
                probs_list.append(probs)
                targets_list.append(yb.numpy().squeeze())

        all_probs = np.concatenate(probs_list)
        all_targets = np.concatenate(targets_list)
        all_preds = (all_probs > 0.5).astype(int)

        # compute metrics
        accuracy = (all_preds == all_targets).mean()
        precision = precision_score(all_targets, all_preds, zero_division=0)
        recall = recall_score(all_targets, all_preds, zero_division=0)
        f1 = f1_score(all_targets, all_preds, zero_division=0)
        cm = confusion_matrix(all_targets, all_preds)

        print(f"Epoch {epoch+1}/{epochs} — train_loss: {train_epoch_loss:.4f}  val_f1: {f1:.4f}")
        print(f"Acc: {accuracy:.4f}, Prec: {precision:.4f}, Rec: {recall:.4f}")
        print("Confusion matrix:\n", cm)

        wandb.log({
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "confusion matrix": cm,
        })

        # -------- early stopping logic --------

        if monitor == "f1":
            current = f1
        elif monitor == "precision":
            current = precision
        elif monitor == "recall":
            current = recall
        elif monitor == "accuracy":
            current = accuracy
        else:
            raise ValueError("Unknown monitor metric")

        improved = (current > best_metric) if mode == "max" else (current < best_metric)
        if improved:
            best_metric = current
            wait = 0
            torch.save(model.state_dict(), best_model_path)   # save best weights
            print(f"  New best {monitor}: {best_metric:.4f}. Saved model.")
        else:
            wait += 1
            print(f"  No improvement for {wait} epoch(s).")

        if wait >= patience:
            print("Early stopping triggered. Restoring best model.")
            break


    model.load_state_dict(torch.load(best_model_path))

    model.load_state_dict(torch.load(best_model_path))
    model.eval()

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for xb, yb in val_loader:
            logits = model(xb)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).long()
            all_preds.append(preds.cpu())
            all_targets.append(yb.unsqueeze(1).cpu())

    # Convert to numpy arrays
    all_preds = torch.cat(all_preds).squeeze().numpy()
    all_targets = torch.cat(all_targets).squeeze().numpy()

    # --- Compute metrics ---
    accuracy = accuracy_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds, zero_division=0)
    recall = recall_score(all_targets, all_preds, zero_division=0)
    f1 = f1_score(all_targets, all_preds, zero_division=0)
    cm = confusion_matrix(all_targets, all_preds)

    # --- Print results ---
    print("Best Model Validation Metrics:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
else:
    print("Nothing to do.")


In [ ]:
if run == 2:
    # --- Create all possible combinations
    param_combinations = list(itertools.product(*config.values()))

    # --- Iterate through all combinations
    for combo in param_combinations:
        params = dict(zip(config.keys(), combo))
        print(f"\nRunning grid search with params: {params}")

        wandb.init(project="ZNEUS_R&B", config=params, name="GC_grid_search")
        config = wandb.config

        train_loader = DataLoader(train_ds, batch_size= config.batch_size, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size= config.batch_size, shuffle=False)

        model = MagicalNet(in_dim=in_features, hidden_dim=config.hidden_dim, dropout_p=config.dropout_p)
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([config.pos_weight]))
        optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

        # --- Training loop (your original)
        for epoch in range(config.epochs):
            model.train()
            running_loss = 0.0

            for xb, yb in train_loader:
                xb, yb = xb, yb.unsqueeze(1)
                optimizer.zero_grad()
                logits = model(xb)
                loss = criterion(logits, yb)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * xb.size(0)

            epoch_loss = running_loss / len(train_loader.dataset)
            wandb.log({"train_loss": epoch_loss, "epoch": epoch + 1})
            #print(f"Epoch {epoch+1}/{config.epochs} — loss: {epoch_loss:.4f}")

        # --- Validation accuracy check
        model.eval()
        probs_list = []
        targets_list = []

        with torch.no_grad():
            for xb, yb in val_loader:
                logits = model(xb)
                probs = torch.sigmoid(logits).cpu().numpy().squeeze()
                probs_list.append(probs)
                targets_list.append(yb.numpy().squeeze())

        all_probs = np.concatenate(probs_list)
        all_targets = np.concatenate(targets_list).astype(int)
        all_preds = (all_probs > 0.5).astype(int)

        accuracy = accuracy_score(all_targets, all_preds)
        precision = precision_score(all_targets, all_preds)
        recall = recall_score(all_targets, all_preds)
        f1 = f1_score(all_targets, all_preds)
        cm = confusion_matrix(all_targets, all_preds)

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
        #print("Confusion matrix:\n", cm)

        wandb.log({
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "confusion_matrix": cm,
        })
else:
    print("No grid search.\n"
          " 🥺\n"
          "👉👈\n")

In [ ]:
if run == 3:
    # --- Hyperparameter grid
    param_combinations = list(itertools.product(*config.values()))


    # --- Grid search
    for combo in param_combinations:
        params = dict(zip(config.keys(), combo))
        print(f"\nRunning grid search with params: {params}")


        run = wandb.init(project="ZNEUS_R&B",name = "GC_Grid_search_w_es", config=params, reinit=True)
        config = wandb.config

        train_loader = DataLoader(train_ds, batch_size= config.batch_size, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size= config.batch_size, shuffle=False)

        # build model / criterion / optimizer
        model = MagicalNet(in_dim=in_features, hidden_dim=config.hidden_dim, dropout_p=config.dropout_p)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

        # early stopping state
        best_epoch = -1
        best_metrics = None

        # training + validation per epoch (so we can early-stop)
        for epoch in range(config.epochs):
            # -------- training --------
            model.train()
            running_loss = 0.0
            for xb, yb in train_loader:
                xb, yb = xb, yb.unsqueeze(1)
                optimizer.zero_grad()
                logits = model(xb)
                loss = criterion(logits, yb)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * xb.size(0)

            train_epoch_loss = running_loss / len(train_loader.dataset)

            # -------- validation --------
            model.eval()
            probs_list = []
            targets_list = []
            with torch.no_grad():
                for xb, yb in val_loader:
                    logits = model(xb)
                    probs = torch.sigmoid(logits).cpu().numpy().squeeze()
                    probs_list.append(probs)
                    targets_list.append(yb.numpy().squeeze())

            all_probs = np.concatenate(probs_list)
            all_targets = np.concatenate(targets_list).astype(int)
            all_preds = (all_probs > 0.5).astype(int)

            # compute metrics
            prec = precision_score(all_targets, all_preds, zero_division=0)
            rec = recall_score(all_targets, all_preds, zero_division=0)
            f1 = f1_score(all_targets, all_preds, zero_division=0)
            acc = accuracy_score(all_targets, all_preds)
            cm = confusion_matrix(all_targets, all_preds)

            if monitor == "f1":
                current = f1
            elif monitor == "precision":
                current = prec
            elif monitor == "recall":
                current = rec
            elif monitor == "accuracy":
                current = acc
            else:
                raise ValueError(f"Unknown monitor metric: {monitor}")

            # logging
            wandb.log({
                "train_loss": train_epoch_loss,
                "accuracy": acc,
                "precision": prec,
                "recall": rec,
                "f1_score": f1,
                "epoch": epoch + 1
            })

            print(f"Epoch {epoch+1}/{config.epochs} — train_loss: {train_epoch_loss:.4f}  val_f1: {f1:.4f}")

            # early stopping decision
            improved = (current > best_metric) if mode == "max" else (current < best_metric)
            if improved:
                best_metric = current
                wait = 0
                best_epoch = epoch + 1
                best_metrics = {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "confusion_matrix": cm}
                torch.save(model.state_dict(), best_model_path)
                print(f"  New best {monitor}: {best_metric:.4f} (saved)")
            else:
                wait += 1
                print(f"  No improvement for {wait} epoch(s) (patience={patience})")

            if wait >= patience:
                print("Early stopping triggered.")
                break

        if best_metrics is not None:
            model.load_state_dict(torch.load(best_model_path, map_location="cpu"))
            model.eval()

            probs_list = []
            targets_list = []
            with torch.no_grad():
                for xb, yb in val_loader:
                    logits = model(xb)
                    probs = torch.sigmoid(logits).cpu().numpy().squeeze()
                    probs_list.append(probs)
                    targets_list.append(yb.numpy().squeeze())

            all_probs = np.concatenate(probs_list)
            all_targets = np.concatenate(targets_list).astype(int)
            all_preds = (all_probs > 0.5).astype(int)

            prec = precision_score(all_targets, all_preds, zero_division=0)
            rec = recall_score(all_targets, all_preds, zero_division=0)
            f1 = f1_score(all_targets, all_preds, zero_division=0)
            acc = accuracy_score(all_targets, all_preds)
            cm = confusion_matrix(all_targets, all_preds)

            # log final best-run metrics
            wandb.log({
                "best_val_accuracy": acc,
                "best_val_precision": prec,
                "best_val_recall": rec,
                "best_val_f1": f1
            })
            print("BEST validation metrics (loaded best model):")
            print(f"Accuracy:  {acc:.4f}")
            print(f"Precision: {prec:.4f}")
            print(f"Recall:    {rec:.4f}")
            print(f"F1-score:  {f1:.4f}")
            print("Confusion matrix:\n", cm)
        else:
            print("No improvement recorded during run; no best model saved.")
else:
    print("Why am I here?. Just to suffer!")



In [ ]:
wandb.finish()

In [ ]:
# model = MagicalNet(in_dim=in_features, hidden_dim=64)
#
# # Load the saved weights
# model.load_state_dict(torch.load('model_87.pt'))
# model.eval()
#
# # Example input — your real data goes here
# new_input = [[1.46773678e+00, 2.22794825e+00 , 2.22794825e+00 , 1.68000000e-01, 1.01100000e-01,
#               1.68377205e+00, 2.20081444e+005, -1.79772516e-01, 2.31902800e+02]]
#
# # Convert to tensor
# x = torch.tensor(new_input, dtype=torch.float32)
#
#
# with torch.no_grad():
#     logit = model(x)
#     prob = torch.sigmoid(logit).item()      # convert to probability
#     pred = int(prob > 0.5)
#
# if pred == 1:
#     print(f"Prediction: Class 1 (probability: {prob:.3f})")
# else:
#     print(f"Prediction: Class 0 (probability: {prob:.3f})")

In [ ]:
# idx = DS_target[DS_target['class'] == 0].index[0]
#
# feature_row = DS_feature_scaled.loc[idx, ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fAlpha', 'fDist']]
# target_val = DS_target.loc[idx, 'class']
#
# print("Features:", feature_row.values)
# print("Target:", target_val)
